# Make postISRCCD from raw for Spectractor


- work with Weakly_2022_39
- use jupyter kernel LSST : **lsst_distrib_2022_39**



- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2022/12/08
- last update : 2022/12/09



In [1]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_01/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-5.1.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_01/Spectractor 	setup


In [2]:
! echo $IMAGE_DESCRIPTION
! eups list -s lsst_distrib

w_2023_01
   g754a7f0350+935911ccad 	current w_2023_01 setup


In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates

import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


from astropy.io import fits




In [4]:
# Assembly task
# https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/isrTask.py

from lsst.ip.isr.assembleCcdTask import (AssembleCcdConfig, AssembleCcdTask)
from lsst.ip.isr.isrTask import (IsrTask, IsrTaskConfig)

#https://github.com/lsst/ip_isr/blob/main/python/lsst/ip/isr/overscan.py
from lsst.ip.isr import  OverscanCorrectionTaskConfig, OverscanCorrectionTask

In [5]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [6]:
import lsst.daf.butler as dafButler

# Configuration

In [7]:
FLAG_PLOT=False

In [8]:
#DATE = 20221207
DATE = 20230117
FILTER="empty-holo4_003"
#filename_visits = f"visitdispersers_{DATE}_filt_{FILTER}.list"
filename_visits = f"all_visitdispersers/{DATE}/visitdispersers_{DATE}_filt_{FILTER}.list"

In [9]:
top_path_out="my_postisrccd_img"
path_out=f"{top_path_out}/{DATE}"

In [10]:
if not os.path.exists(path_out):
    os.makedirs(path_out)

# read list of exposures

- generated by ListOfExposures-hologram.ipynb

In [11]:
df = pd.read_csv(filename_visits, header=None,skiprows = 1, sep=' ',names=["date","seq"])
df

,date,seq
0,20230117,239
1,20230117,243
2,20230117,244
3,20230117,248
4,20230117,249
...,...,...
78,20230117,670
79,20230117,678
80,20230117,679
81,20230117,687


In [12]:
for index,row in df.iterrows():
    exposure_selected =row["date"]*100000+row["seq"]
    print(exposure_selected)

2023011700239
2023011700243
2023011700244
2023011700248
2023011700249
2023011700265
2023011700266
2023011700275
2023011700276
2023011700285
2023011700286
2023011700297
2023011700298
2023011700316
2023011700317
2023011700325
2023011700326
2023011700334
2023011700335
2023011700348
2023011700349
2023011700363
2023011700364
2023011700375
2023011700376
2023011700389
2023011700390
2023011700411
2023011700412
2023011700420
2023011700421
2023011700429
2023011700430
2023011700438
2023011700439
2023011700447
2023011700448
2023011700457
2023011700458
2023011700467
2023011700468
2023011700476
2023011700477
2023011700486
2023011700487
2023011700496
2023011700497
2023011700505
2023011700506
2023011700514
2023011700515
2023011700524
2023011700525
2023011700534
2023011700535
2023011700543
2023011700544
2023011700552
2023011700553
2023011700561
2023011700562
2023011700570
2023011700571
2023011700589
2023011700590
2023011700598
2023011700599
2023011700607
2023011700608
2023011700616
2023011700617
202301

## Select flags options

In [13]:
FLAG_ROTATE_IMG = True
FLAG_TRANSFORM = True

## Transformations
astropy scale transformations

In [14]:
transform = AsinhStretch() + PercentileInterval(99.)
#transform = PercentileInterval(98.)

## Butler

In [15]:
#repo = '/sdf/group/rubin/repo/main'
repo="/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [16]:
collection='LATISS/raw/all'

In [17]:
# configuration
isr_config =  IsrTaskConfig()

In [18]:
isr_config.doDark = False
isr_config.doFlat =  False
isr_config.doFringe = False
isr_config.doDefect = True
isr_config.doLinearize = False
isr_config.doCrosstalk =  False
isr_config.doSaturationInterpolation = False
isr_config.overscan.fitType: 'MEDIAN_PER_ROW'
isr_config.doBias: True


In [19]:
isr_task = IsrTask(config=isr_config)

In [20]:
calibType = 'bias'
physical_filter = 'empty~empty'
cameraName = 'LATISS'
# Collection name containing the verification outputs.
calibCollections = ['LATISS/calib','LATISS/raw/all',
                   'LATISS/calib/DM-28636',
'LATISS/calib/DM-28636/curated/19700101T000000Z',
'LATISS/calib/DM-28636/curated/20180101T000000Z',
'LATISS/calib/DM-28636/unbounded',
'LATISS/calib/DM-33875',
'LATISS/calib/DM-36484/bias.20221005a',
'LATISS/calib/DM-36484/biasGen.20221005a',
'LATISS/calib/DM-36484/biasGen.20221005a/20221006T000101Z',
'LATISS/calib/DM-36484/dark.20221006a',
'LATISS/calib/DM-36484/darkGen.20221005a',
'LATISS/calib/DM-36484/darkGen.20221005a/20221006T222501Z',
'LATISS/calib/DM-36484/darkGen.20221006a',
'LATISS/calib/DM-36484/darkGen.20221006a/20221006T222921Z',
'LATISS/calib/DM-36484/flat-SDSSg.20221006a',
'LATISS/calib/DM-36484/flat-SDSSi.20221006a',
'LATISS/calib/DM-36484/flat-SDSSr.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSg.20221006a/20221007T002703Z',
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSi.20221006a/20221007T003732Z',
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSiD.20221006a/20221007T004708Z',
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a',
'LATISS/calib/DM-36484/flatGen-SDSSr.20221006a/20221006T233657Z',
'LATISS/calib/DM-36484/verifyBias.20221005a',
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T000747Z',
'LATISS/calib/DM-36484/verifyBias.20221005a/20221006T213237Z',
'LATISS/calib/DM-36484/verifyBias.20221005b',
'LATISS/calib/DM-36484/verifyBias.20221005b/20221019T205236Z',
'LATISS/calib/DM-36484/verifyDark.20221006a',
'LATISS/calib/DM-36484/verifyDark.20221006a/20221006T224403Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSg.20221006a/20221007T003418Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSi.20221006a/20221007T004423Z',
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a',
'LATISS/calib/DM-36484/verifyFlat-SDSSr.20221006a/20221006T234341Z',
'LATISS/calib/DM-36719',
'LATISS/calib/DM-36719/bias.20221107',
'LATISS/calib/DM-36719/biasGen.20221107a',
'LATISS/calib/DM-36719/biasGen.20221107a/20221107T205127Z',
'LATISS/calib/DM-36719/biasGen.20221107b',
'LATISS/calib/DM-36719/biasGen.20221107b/20221107T213306Z',
'LATISS/calib/DM-36719/dark.20221107',
'LATISS/calib/DM-36719/darkGen.20221107a',
'LATISS/calib/DM-36719/darkGen.20221107a/20221107T223409Z',
'LATISS/calib/DM-36719/flat-SDSSg.20221107',
'LATISS/calib/DM-36719/flat-SDSSi.20221107',
'LATISS/calib/DM-36719/flat-SDSSr.20221107',
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSg.20221107a/20221108T002737Z',
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSi.20221107a/20221108T005202Z',
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a',
'LATISS/calib/DM-36719/flatGen-SDSSr.20221107a/20221107T235401Z',
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a',
'LATISS/calib/DM-36719/ptcGen-SDSSr.20221107a/20221108T180421Z',
'LATISS/calib/DM-36719/verifyBias.20221107b',
'LATISS/calib/DM-36719/verifyBias.20221107b/20221107T220410Z',
'LATISS/calib/DM-36719/verifyDark.20221107a',
'LATISS/calib/DM-36719/verifyDark.20221107a/20221107T232823Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSg.20221107a/20221108T004225Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T012110Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSi.20221107a/20221108T014950Z',
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a',
'LATISS/calib/DM-36719/verifyFlat-SDSSr.20221107a/20221108T000940Z',
'LATISS/calib/unbounded','LATISS/defaults','LATISS/raw/all']

In [21]:
butler = dafButler.Butler(repo, collections=calibCollections)
camera = butler.get('camera', instrument=cameraName)
bias = butler.get('bias',instrument=cameraName,detector=0)
defects = butler.get('defects',instrument=cameraName,detector=0)


In [22]:
if FLAG_PLOT:
    for index,row in df.iterrows():
        exposure_selected =row["date"]*100000+row["seq"]



        raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
        meta = raw_img.getMetadata()
        md = meta.toDict()
    
        the_object = md['OBJECT']
        the_am= md['AMSTART']
        the_filter=md['FILTER']
    
        #fast ISR 
        isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
        rotated_array = isr_img.exposure.image.array[::-1,::-1] #rotate the array 180 degrees
        #np.flip(np.flip(a, 1), 0)

    
        fig = plt.figure(figsize=(12,10))
        afw_display = afwDisplay.Display(frame=fig)
        afw_display.scale('linear', 'zscale',None)
        the_title = f"{index} :: isr exposure : {exposure_selected}, target={the_object}, airmass={the_am:.2f}, filter={the_filter}"
        afw_display.mtv(isr_img.exposure.image,title=the_title)

In [23]:
# Save in files

In [24]:
for index,row in df.iterrows():
    
    exposure_selected =row["date"]*100000+row["seq"]



    raw_img= butler.get('raw', dataId={'exposure': exposure_selected, 'instrument': 'LATISS', 'detector': 0}, collections = collection)
    isr_img = isr_task.run(raw_img,bias=bias,defects=defects)
    
    arr=isr_img.exposure.image.array
    # 180 degree rotation
    rotated_array = arr[::-1,::-1] #rotate the array 180 degrees
    
    
    meta = raw_img.getMetadata()
    md = meta.toDict()

    the_object = md['OBJECT']
    the_am= md['AMSTART']
    the_filter=md['FILTER']

    filename_out = f"exposure_{exposure_selected}_pseudo-postisrccd.fits"
    fullfilename_out=os.path.join(path_out,filename_out)
    
    print(filename_out)
    
    hdr = fits.Header()
    
    for key,value in md.items():
        hdr[str(key)]=value
        
    # need this    
    hdr["AMEND"] = hdr["AMSTART"]
    
    # be aware weather data may be missing
    
        
    hdu = fits.PrimaryHDU(data=rotated_array,header=hdr)  # with headers
    #hdu = fits.PrimaryHDU(data=all_my_raw_array[idx])
    
    hdul = fits.HDUList([hdu])
    
    hdul.writeto(fullfilename_out,overwrite=True)
    





exposure_2023011700239_pseudo-postisrccd.fits


exposure_2023011700243_pseudo-postisrccd.fits


exposure_2023011700244_pseudo-postisrccd.fits


exposure_2023011700248_pseudo-postisrccd.fits


OSError: [Errno 28] No space left on device

In [ ]:
rotated_array.shape